In [1]:
import datetime
import numpy as np
from tensorflow.keras.backend import conv2d
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

x_train = np.load('x_train.npy').astype(np.float32)
y_train = np.load('y_train.npy').astype(np.float32)
x_val = np.load('x_val.npy').astype(np.float32)
y_val = np.load('y_val.npy').astype(np.float32)

plt.subplot(2, 1, 1)
plt.title(str(y_train[0]))
plt.imshow(x_train[0].reshape((26, 34)), cmap='gray')
plt.subplot(2, 1, 2)
plt.title(str(y_val[4]))
plt.imshow(x_val[4].reshape((26, 34)), cmap='gray')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

#data generator
train_generator = train_datagen.flow(
    x=x_train, y=y_train,
    batch_size=32,
    shuffle=True
)

val_generator = val_datagen.flow(
    x=x_val, y=y_val,
    batch_size=32,
    shuffle=False
)

inputs = Input(shape=(26, 34, 1)) #26*34 gray scale image

#커널수를 변경하며 세번 
net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
net = MaxPooling2D(pool_size=2)(net)#차원 축소

net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Flatten()(net)

net = Dense(512)(net)
net = Activation('relu')(net)
net = Dense(1)(net)      #value 0~1
outputs = Activation('sigmoid')(net)

#model definition
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

#train
model.fit_generator(
    train_generator, epochs=50, validation_data=val_generator,
    callbacks=[
        ModelCheckpoint('%s.h5' % (start_time), monitor='val_acc', save_best_only=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05)
    ]
)

Epoch 1/50
80/81 [============================>.] - ETA: 0s - loss: 0.5402 - acc: 0.7334
Epoch 00001: val_acc improved from -inf to 0.87847, saving model to 2022_03_08_18_29_49.h5
81/81 [==============================] - 5s 67ms/step - loss: 0.5370 - acc: 0.7351 - val_loss: 0.3025 - val_acc: 0.8785
Epoch 2/50
80/81 [============================>.] - ETA: 0s - loss: 0.3058 - acc: 0.8743
Epoch 00002: val_acc improved from 0.87847 to 0.92708, saving model to 2022_03_08_18_29_49.h5
81/81 [==============================] - 6s 79ms/step - loss: 0.3042 - acc: 0.8751 - val_loss: 0.1787 - val_acc: 0.9271
Epoch 3/50
80/81 [============================>.] - ETA: 0s - loss: 0.1952 - acc: 0.9229
Epoch 00003: val_acc improved from 0.92708 to 0.95486, saving model to 2022_03_08_18_29_49.h5
81/81 [==============================] - 8s 94ms/step - loss: 0.1957 - acc: 0.9227 - val_loss: 0.1147 - val_acc: 0.9549
Epoch 4/50
80/81 [============================>.] - ETA: 0s - loss: 0.1727 - acc: 0.9397
Epoch

Epoch 31/50
80/81 [============================>.] - ETA: 0s - loss: 0.0168 - acc: 0.9949
Epoch 00031: val_acc did not improve from 1.00000

Epoch 00031: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
81/81 [==============================] - 12s 144ms/step - loss: 0.0166 - acc: 0.9950 - val_loss: 0.0575 - val_acc: 0.9722
Epoch 32/50
80/81 [============================>.] - ETA: 0s - loss: 0.0258 - acc: 0.9926
Epoch 00032: val_acc did not improve from 1.00000
81/81 [==============================] - 8s 102ms/step - loss: 0.0255 - acc: 0.9927 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 33/50
80/81 [============================>.] - ETA: 0s - loss: 0.0130 - acc: 0.9949
Epoch 00033: val_acc did not improve from 1.00000
81/81 [==============================] - 8s 102ms/step - loss: 0.0129 - acc: 0.9950 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 34/50
80/81 [============================>.] - ETA: 0s - loss: 0.0118 - acc: 0.9945
Epoch 00034: val_acc did not improve from 1.